In [1]:
import pandas as pd
import ast

In [2]:
data = pd.read_csv('data/CombinedContinuousData.csv')
data

,QuestionText,QType,Answer,AnswerChoices,Year,Tags,Description
0,How many Federally Insured Commercial Banks in...,continuous,0.420490,"{'type': 'continuous', 'scale': {'max': 5500, ...",2018,['business'],NaN
1,When will an AI system score an impressive def...,continuous,0.376395,"{'type': 'continuous', 'scale': {'max': '2030-...",2019,"['comp sci ai and machinelearning', 'computin...",[Starcraft 2](http://us.battle.net/sc2/en/) (S...
2,When will programs write programs for us?,continuous,0.408719,"{'type': 'continuous', 'scale': {'max': '2050-...",2021,"['technological advances', 'comp sci ai and m...","Since the early days of ""machine code"" program..."
3,When will the first exaflop performer appear?,continuous,0.849452,"{'type': 'continuous', 'scale': {'max': '2025-...",2022,['computing computers'],NaN
4,When will the first law concerning artificial ...,continuous,0.260335,"{'type': 'continuous', 'scale': {'max': '2060-...",2021,"['computing ai', 'law legislation', 'series ...","As of now, there are to the author's knowledge..."
...,...,...,...,...,...,...,...
2532,What will be the 7-day moving average of daily...,continuous,0.168667,"{'max': 15000, 'min': 0, 'deriv_ratio': 1}",2022,"['Novel Coronavirus (Covid-19)', 'Tournament -...",Following the decline of Omicron wave in Virgi...
2533,What will be the 7-day moving average of daily...,continuous,0.214533,"{'max': 15000, 'min': 0, 'deriv_ratio': 1}",2022,"['Novel Coronavirus (Covid-19)', 'Tournament -...",Following the decline of Omicron wave in Virgi...
2534,What will be the 7-day moving average of curre...,continuous,0.069975,"{'max': 4000, 'min': 0, 'deriv_ratio': 1}",2022,"['Novel Coronavirus (Covid-19)', 'Tournament -...","Nationwide, new COVID-19 hospitalizations have..."
2535,What will be the 7-day moving average of curre...,continuous,0.121100,"{'max': 4000, 'min': 0, 'deriv_ratio': 1}",2022,"['Novel Coronavirus (Covid-19)', 'Tournament -...","Nationwide, new COVID-19 hospitalizations have..."


In [3]:
startFrom, endAt = 0, 2537

In [4]:
questions = list(data['QuestionText'])

In [5]:
maxes = []
mins = []
for choice in list(data['AnswerChoices']):
    choice = ast.literal_eval(choice)
    if 'scale' in choice:
        maxes.append(choice['scale']['max'])
        mins.append(choice['scale']['min'])
    else:
        maxes.append(choice['max'])
        mins.append(choice['min'])

In [6]:
from gpt4all import GPT4All
from tqdm import tqdm

def saveAnswers(questions, start, end):
    predictions = pd.DataFrame()
    predictions['predictions'] = questions
    predictions.to_csv(f'data/WizardContinuous{start}-{end}.csv')

model = GPT4All("wizardlm-13b-v1.2.Q4_0.gguf")
outputs = []
for i in tqdm(range(startFrom, endAt)):
    with model.chat_session():
        re = model.generate(f"### Instruction: \nAnswer the question below only with a number between {mins[i]} and {maxes[i]}. Feel free to hallucinate\n ### Question: \n{questions[i]} \n ### Response:\n", temp=0.5)
        outputs.append(re)
        if i % 100 == 0:
            saveAnswers(outputs, startFrom, i)

100%|███████████████████████████████████████████████████████████████████████████| 2537/2537 [31:25:33<00:00, 44.59s/it]


In [7]:
saveAnswers(outputs, 0, 2536)